https://keras.io/examples/vision/token_learner/

In [ ]:
import keras 
from keras import layers
from keras import ops
from tensorflow import data as tf_data

from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import math